In [ ]:
# Role Seller for APAD Project 1

# Author: Tianyi (Kelly) Zhang
#         Shikha Singh

# Creation Date: 07/21/2019

# Major: MSITM

In [13]:
import Ipynb_importer
import sqlite3
import Database
from datetime import date, datetime
import csv
import glob

In [12]:
class Seller:
    image_dir = "Images/"
    export_dir = "Exports/"
    import_dir = "ImportFiles/"

    @staticmethod
# This function add the clothing items in the database
# The items are available for renting from the day they are added in the system. 
# So we have used date.today() function to populate Available_From unless specified by seller explicitly
    def add_item(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From=None) :

        db = Database.Database.initialize()
        cursor = db.cursor()
        try:
#             Checking the images in the specide image directory
            for image in glob.glob(Seller.image_dir+'/*'): 
#             Checking if the image specified by the seller is available in the image directory
                if (str(image).replace(Seller.image_dir,'')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
                else:
                    Print("The image file specified is not found. Please ensure the file/folder name is correct")
                    return None
        except:
            print("Could not read file. Check your file name and folder path")
            
        if Available_From == "" or Available_From is None:
            Available_From = date.today()
            
        cursor.execute(
            '''Insert into Inventory_Items(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From) values(?,?,?,?,?,?,?,?,?,?,?)''',
            (Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From))
        db.commit()

        Database.Database.close_connection()

    @staticmethod
    def remove_item(Item_ID):
#         This function removes the inventory item from the database
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Delete from Inventory_Items where Item_ID == ?''',(str(Item_ID)))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def update_item(Item_ID,Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From):
#        This function updates any/all attributes of the inventory item in the database
        db = Database.Database.initialize()
        cursor = db.cursor()
        
        try:
            for image in glob.glob(Seller.image_dir + '/*'):
                if (str(image).replace(Seller.image_dir, '')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
        except:
            print("Could not read file. Check your file name and folder path")
            
        cursor.execute('''Update Inventory_Items 
        set Brand_Name= ?,
        Type = ? ,
        Size =?,
        Gender = ?,
        Original_Price =?,
        Rental_Price =?,
        Owner_ID=?,
        Location=?,
        Cloth_Image=?,
        Deposit=?,
        Available_From=? where Item_ID = ?''',(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From,Item_ID))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def ship_item(Order_ID):
# This function will be called when the Seller clicks on ship item button. 
# This will change the status of the order to "Order Shipped"
# This will also update the shipping date to current date

        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                set Order_Status = 'Order Shipped',
                    Shipping_Date = ?
                where Order_ID = ?''', (date.today(),Order_ID))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def refund_deposit(Order_ID):
# This function will be called when the Seller clicks on refund deposit button. 
# This function will update the order status to "Return Received(Deposit Refunded)" 
# This function will update the item's Available From date as current date
# This function will execute when the return has been initited by the buyer(order status is "return Initiated")
# and return date is current day or a future date


        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                    set Order_Status = 'Return Received(Deposit Refunded)'
                    where Order_ID = ? and Return_Date >= ? and Order_Status = "Return Initiated" ''', (str(Order_ID),date.today()))
        
        cursor.execute('''select Item_ID from Orders where Order_ID = ?''',(Order_ID,))
        for row in cursor.fetchall():
            item = row[0]
        cursor.execute('''Update Inventory_Items 
                    set Available_From =? where Item_ID = ?''', (date.today(),item))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def withhold_deposit(Order_ID):
# This function will be called when the Seller clicks on withhold deposit button. 
# This function will update the order status to "Return Not Received(Deposit Withheld)" 
# This function will execute when the return has NOT been initited by the buyer(order status is NOT "return Initiated")
# and return date is less that current date i.e. a past date
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                        set Order_Status = 'Return Not Received(Deposit Withheld)'
                        where Order_ID = ? and Return_Date < ? and Order_Status != "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def bulk_item_upload(filename):
# This function add the clothing items in the database in bulk
# This function reads from the csv file provided by the Seller and adds each row of the csv as a new inventory item
        try:
#             Checking the file in the specide import file directory
            for files in glob.glob(Seller.import_dir + '/*'):
#             Checking if the file specified by the seller is available in the directory
                if (str(files).replace(Seller.import_dir, '')) == filename:
                    print(str(files).replace(Seller.import_dir, ''))
                    with open(Seller.import_dir + '/'+filename, 'r') as inputfile:
                        items = csv.DictReader(inputfile)

                        for item in items:
                            if item['AvailableFrom'] == '':
                                item['AvailableFrom'] = date.today()
                            Seller().add_item(item['Brand'],item['Type'],item['Size'],item['Gender'],item['OriginalPrice'],item['RentalPrice'],item['OwnerID'],item['Location'],item['ClothImage'],item['Deposit'],item['AvailableFrom'])
        except:
            print("Could not read file. Check your file name and folder path")
    
    @staticmethod
    def all_orders():
# This function takes fetches all orders in the system along with the item, buyer and seller details
# This function creates a csv file in the specified directory and writes the fetch result in the file
# The file is created with a date time stamp so that every time this function is executed a new file is created
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID ''')

        with open(Seller.export_dir+'/'+"Orders_export_"+ str(datetime.now()) + ".csv", "w", newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([i[0] for i in cursor.description])  # write headers
            csv_writer.writerows(cursor)
        Database.Database.close_connection()
        
    @staticmethod
    def orders_per_seller(Seller_ID):
# This function fetches the orders from a system along with the item and buyer for a specified seller
# This function creates a csv file in the specified directory and writes the fetch result in the file
# The file is created with the seller's username and date time stamp so that every time this function is executed a new file is created
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''select User_Name from Users where User_ID = ?''',(Seller_ID,))
        for row in cursor.fetchall():
            seller = row[0]
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                        u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                        i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender"
                                        from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID 
                                        where i.Owner_ID = ? ''', (Seller_ID,))

        try:
            with open(Seller.export_dir + '/' + seller + "'s_Orders_export_" + str(datetime.now()) + ".csv", "w", newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                csv_writer.writerows(cursor)
        except:
            print("Could not write to the file")
        Database.Database.close_connection()
        
    @staticmethod
    def buyer_order_history(User_ID):
# This function fetches the orders for a user
        db = Database.Database.initialize()    
        cursor = db.cursor()
        cursor.execute('''select * from Orders where User_ID = ?''',(User_ID,))
        for row in cursor.fetchall():
            print(row)
        Database.Database.close_connection()

In [13]:
Seller.buyer_order_history(1)

NameError: name 'Database' is not defined

In [16]:
Seller.bulk_item_upload('ItemInventory.csv')

Could not read file. Check your file name and folder path
